### Challenge 38: Offline dictionary attack on simplified SRP

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [66]:
from Crypto.Util import number
from Crypto.Random import random
from Crypto.Hash.SHA256 import SHA256Hash

import cryptopals as cp

<div class="alert alert-block alert-info">

**S**

<div class="alert alert-block alert-info">
    
x = SHA256(salt|password)   
    
v = g**x % n

</div>
    
**C->S**
    
<div class="alert alert-block alert-info">
    
I, A = g**a % n

</div>

**S->C**

<div class="alert alert-block alert-info">
  
salt, B = g**b % n, u = 128 bit random number

</div>
  
**C**

<div class="alert alert-block alert-info">
    
x = SHA256(salt|password)
    
S = B**(a + ux) % n
    
K = SHA256(S)

</div>
  
**S**
    
<div class="alert alert-block alert-info">

S = (A * v ** u)**b % n
    
K = SHA256(S)

</div>
  
**C->S**

<div class="alert alert-block alert-info">
    
Send HMAC-SHA256(K, salt)

</div>

**S->C**

<div class="alert alert-block alert-info">
  
Send "OK" if HMAC-SHA256(K, salt) validates

</div>
    
Note that in this protocol, the server's "B" parameter doesn't depend on the password (it's just a Diffie Hellman public key).

Make sure the protocol works given a valid password.

Now, run the protocol as a MITM attacker: pose as the server and use arbitrary values for b, B, u, and salt.

Crack the password from A's HMAC-SHA256(K, salt).

</div>

---

In [67]:
# Code modified from Challenge #37 to implement the simplified SRP protocol 

class session:
    
    def __init__(self, user_name, salt, A, B, p, u):
        
        self.salt = salt
        self.user_name = user_name
        self.A = A
        self.B = B
        self.p = p
        self.u = u
        
class server:

    class password_entry:
    
        def __init__(self, salt, v):
        
            self.salt = salt
            self.v = v
    
    g = 2
    k = 3
    N = int('e7bd694dded5483a8083fe55c063b0180887542ef644e655d13d40bd1cd808f1'\
            'a9bca3c012478f7cf744222ccf7580036af82c589375e2dddffb6d698e177884'\
            '2594a439e123cd0fc91fbb017bfa7ff312c270d1b0f01def9f31db62872d3cb4'\
            'ed7d62448f13fb1742933223753be04ff20ce0903718f54406305cf6617212bf'\
            '27db2986654a09fbfe6b56445b1af6805384d483be70fc2f6c96a87ebd47e28a'\
            'ffb1b28b0c3b416fa49e4cef5e1a55e7268431c67d3a80daa0a16ddfe6f335e7'\
            'ecd98587a2348bf2e1cde93d42be6f7497fd23d3e74448218f9d0c7143f53773'\
            'c9afeb953fd02623380ffe2ba756668bbf4fdc9c5f56819c09ec055ba01b0447', 16)
    
    def __init__(self):
        
        self.users = {}
        self.current_session = None
    
    def add_user(self, user_name, password):
    
        # Generate random salt (64-bit integer)
        salt = random.randint(0, 2**64-1)
        hashin = (str(salt) + password).encode()
        x = int(SHA256Hash(hashin).digest().hex(), 16)
        v = pow(self.g, x, self.N)
        record = self.password_entry(salt, v)
        self.users[user_name] = record
        
    def initialize_session(self, user_name, A):
        
        salt = self.users[user_name].salt
        v = self.users[user_name].v
        
        u = random.randint(0, 2**128-1) % N
        b = random.randint(0, 2**64-1) % N
        B = pow(g, b, N)        
        
        S_server = pow (A * pow(v, u, N), b, N)
        self.S = S_server
        self.current_session = session(user_name, salt, A, B, b, u)
        self.current_session_key = SHA256Hash(str(S_server).encode()).digest()
        
        return(salt, B, u)
    
    def validate_session(self, C_Proof):
        
        S_Proof = SHA256Hash(self.current_session_key + 
                             str(self.current_session.salt).encode()).digest()
        
        if S_Proof == C_Proof:
            return_data = 'OK'
        else:
            return(random.Random.get_random_bytes(16))
        
        CT = cp.AESEncrypt(return_data, self.current_session_key)
        return(CT)
        

In [68]:
myserver = server()
myserver.add_user('somebody@gmail.com', 'password123')
myserver.add_user('user2', 'goofy')
myserver.add_user('user3', 'pluto')
myserver.add_user('user4@hellokitty.com', 'purrrfect')

---
Now emulate the client code:

---

In [69]:
g = 2
k = 3
N = int('e7bd694dded5483a8083fe55c063b0180887542ef644e655d13d40bd1cd808f1'\
        'a9bca3c012478f7cf744222ccf7580036af82c589375e2dddffb6d698e177884'\
        '2594a439e123cd0fc91fbb017bfa7ff312c270d1b0f01def9f31db62872d3cb4'\
        'ed7d62448f13fb1742933223753be04ff20ce0903718f54406305cf6617212bf'\
        '27db2986654a09fbfe6b56445b1af6805384d483be70fc2f6c96a87ebd47e28a'\
        'ffb1b28b0c3b416fa49e4cef5e1a55e7268431c67d3a80daa0a16ddfe6f335e7'\
        'ecd98587a2348bf2e1cde93d42be6f7497fd23d3e74448218f9d0c7143f53773'\
        'c9afeb953fd02623380ffe2ba756668bbf4fdc9c5f56819c09ec055ba01b0447', 16)

In [70]:
# Generate ephemeral private and public key for a new session

a = random.randint(0, 2**64-1) % N
A = pow(g, a, N)

P = 'purrrfect'

[salt, B, u] = myserver.initialize_session('user4@hellokitty.com', A)

hash_in = (str(salt) + P).encode()
x = int(SHA256Hash(hash_in).digest().hex(), 16)
S_client = pow(B, a + u*x, N)
K_client = SHA256Hash(str(S_client).encode()).digest()

C_Proof = SHA256Hash(K_client + str(salt).encode()).digest()
server_response_CT = myserver.validate_session(C_Proof)

response = cp.strip_PKCS7_pad(cp.AESDecrypt(server_response_CT, K_client))

print(response.decode())

OK


---
Check what happens if we supply a bad password.  The server should just return random gibberish.  

Since PKCS7 padding will be invalid -- this raises an exception.

---

In [71]:
# Generate ephemeral private and public key for a new session

a = random.randint(0, 2**64-1) % N
A = pow(g, a, N)

P = 'Purrrfect'

[salt, B, u] = myserver.initialize_session('user4@hellokitty.com', A)

hash_in = (str(salt) + P).encode()
x = int(SHA256Hash(hash_in).digest().hex(), 16)
S_client = pow(B, a + u*x, N)
K_client = SHA256Hash(str(S_client).encode()).digest()

C_Proof = SHA256Hash(K_client + str(salt).encode()).digest()
server_response_CT = myserver.validate_session(C_Proof)

try:
    response = cp.strip_PKCS7_pad(cp.AESDecrypt(server_response_CT, K_client))
except ValueError:
    print('')

print(response.decode())


OK


---

### **The Attack**

<div class="alert alert-block alert-info">

Now, run the protocol as a MITM attacker: pose as the server and use arbitrary values for b, B, u, and salt.

Crack the password from A's HMAC-SHA256(K, salt).

</div>

This is a little tricker than I thought it was at first.  The principal is simple -- the attacker controls and knows the `salt`, so they can get the user to provide `H(K | Known_Salt)`.  We do _not_ know the client's private key `a`.  To brute force the password that the client used to create their `K`, we emulate adding a new user.  The bit of code I used to do that looks like this:

```python
hash_in = (str(salt) + pwd_guess).encode()
x_guess = int(SHA256Hash(hash_in).digest().hex(), 16)
v_guess = pow(self.g, x_guess, self.N)
S_guess = pow(self.current_session.A * pow(v_guess, self.current_session.u, self.N), 
              self.current_session.p, N)
K_guess = SHA256Hash(str(S_guess).encode()).digest()
C_proof_guess = SHA256Hash(K_guess + str(salt).encode()).digest()
```

Now I just run this code using all of the words in my dictionary and the known salt, and compare the `proof` provided by the client ot what I generate.  If there's a match, we have the correct password.  

For this challenge, I'll just create a simple dictionary that handily includes 20 of the internet's favorit passwords + the password that the client selected.

In [85]:
my_dictionary = ['123456',
                 '12345',
                 '123456789',
                 'Password',
                 'iloveyou',
                 'princess',
                 'rockyou',
                 '1234567',
                 '12345678',
                 'abc123',
                 'Nicole',
                 'Daniel',
                 'babygirl',
                 'monkey',
                 'Jessica',
                 'Lovely',
                 'michael',
                 'Purrrfect',
                 'Ashley',
                 '654321',
                 'Qwerty'
                 ]

In [92]:
# Code modified from Challenge #37 to implement the simplified SRP protocol 
      
class evil_server:

    class password_entry:
    
        def __init__(self, salt, v):
        
            self.salt = salt
            self.v = v
    
    g = 2
    k = 3
    N = int('e7bd694dded5483a8083fe55c063b0180887542ef644e655d13d40bd1cd808f1'\
            'a9bca3c012478f7cf744222ccf7580036af82c589375e2dddffb6d698e177884'\
            '2594a439e123cd0fc91fbb017bfa7ff312c270d1b0f01def9f31db62872d3cb4'\
            'ed7d62448f13fb1742933223753be04ff20ce0903718f54406305cf6617212bf'\
            '27db2986654a09fbfe6b56445b1af6805384d483be70fc2f6c96a87ebd47e28a'\
            'ffb1b28b0c3b416fa49e4cef5e1a55e7268431c67d3a80daa0a16ddfe6f335e7'\
            'ecd98587a2348bf2e1cde93d42be6f7497fd23d3e74448218f9d0c7143f53773'\
            'c9afeb953fd02623380ffe2ba756668bbf4fdc9c5f56819c09ec055ba01b0447', 16)
    
    def __init__(self):
        
        self.users = {}
        self.current_session = None
    
    def add_user(self, user_name, password):
    
        # Generate random salt (64-bit integer)
        salt = random.randint(0, 2**64-1)
        x = int(SHA256Hash((str(salt) + password).encode()).digest().hex(), 16)
        v = pow(self.g, x, self.N)
        record = self.password_entry(salt, v)
        self.users[user_name] = record
        
    def initialize_session(self, user_name, A):
        
        #salt = self.users[user_name].salt
        #v = self.users[user_name].v
        salt = random.randint(0, 2**64-1)
        u = random.randint(0, 2**128-1) % N
        b = random.randint(0, 2**64-1) % N
        B = pow(g, b, N)        
        
        self.current_session = session(user_name, salt, A, B, b, u)
        
        return(salt, B, u)
    
    def validate_session(self, C_proof):
        
        for pwd_guess in my_dictionary:
            
            salt = self.current_session.salt
            hash_in = (str(salt) + pwd_guess).encode()
            x_guess = int(SHA256Hash(hash_in).digest().hex(), 16)
            v_guess = pow(self.g, x_guess, self.N)
            S_guess = pow(self.current_session.A * pow(v_guess, self.current_session.u, self.N), 
                          self.current_session.p, N)
            K_guess = SHA256Hash(str(S_guess).encode()).digest()
            C_proof_guess = SHA256Hash(K_guess + str(salt).encode()).digest()
            
            # print(C_proof.hex())
            # print(C_proof_guess.hex())
            
            if C_proof == C_proof_guess:
                print(f"Gotcha! Your password is `{pwd_guess}`")
                break
                
        return_data = 'OK'
        CT = cp.AESEncrypt(return_data, K_guess)
        return(CT)


In [93]:
myserver = evil_server()
myserver.add_user('somebody@gmail.com', 'password123')
myserver.add_user('user2', 'goofy')
myserver.add_user('user3', 'pluto')
myserver.add_user('user4@hellokitty.com', 'purrrfect')

In [94]:
# Generate ephemeral private and public key for a new session

response = None

a = random.randint(0, 2**64-1) % N
A = pow(g, a, N)

P = 'Purrrfect'

[salt, B, u] = myserver.initialize_session('user4@hellokitty.com', A)

hash_in = (str(salt) + P).encode()
x = int(SHA256Hash(hash_in).digest().hex(), 16)
S_client = pow(B, a + u*x, N)
K_client = SHA256Hash(str(S_client).encode()).digest()

C_Proof = SHA256Hash(K_client + str(salt).encode()).digest()
server_response_CT = myserver.validate_session(C_Proof)

try:
    response = cp.strip_PKCS7_pad(cp.AESDecrypt(server_response_CT, K_client))
    print(response.decode())
except ValueError as error:
    print(f"\n*****\nERROR:  {error}\n*****")



Gotcha! Your password is `Purrrfect`
OK


[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)